In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
import re
import matplotlib.pyplot as plt
import random as rand
import tkinter as tk
from tkinter import filedialog


from pathlib import Path
from sklearn.model_selection import train_test_split

count_x_train = 0
count_y_train = 0
count_x_val = 0
count_y_val = 0

2025-01-29 20:38:44.075736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738179524.091438   53529 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738179524.096043   53529 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 20:38:44.112410: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Crear una ventana oculta de Tkinter
root = tk.Tk()
root.withdraw()  # Ocultar la ventana principal

# Ruta inicial
initial_path = Path("/Users/felixmaral/Desktop/TFG/datasets")

# Seleccionar la primera ruta
print("Selecciona la ruta para 'point_clouds_path'")
point_clouds_path = Path(filedialog.askdirectory(
    title="Selecciona la carpeta para 'point_clouds_path'",
    initialdir=initial_path
))

# Seleccionar la segunda ruta
print("Selecciona la ruta para 'labels_path'")
labels_path = Path(filedialog.askdirectory(
    title="Selecciona la carpeta para 'labels_path'",
    initialdir=initial_path
))

# Mostrar las rutas seleccionadas
print(f"Ruta seleccionada para point_clouds_path: {point_clouds_path}")
print(f"Ruta seleccionada para labels_path: {labels_path}")

Selecciona la ruta para 'point_clouds_path'
Selecciona la ruta para 'labels_path'


KeyboardInterrupt: 

In [ ]:
files_list = sorted(os.listdir(point_clouds_path))
labels_list = sorted(os.listdir(labels_path))

In [ ]:
Y = []

# Reading .label files and adding to Y_DF
for file in labels_list:

    # reading a .label file
    label = np.fromfile(os.path.join(labels_path, file), dtype=np.uint32)
    label = label.reshape((-1))

    # extract the semantic and instance label IDs
    sem_label = label & 0xFFFF  # semantic label in lower half

    Y.append(pd.DataFrame(sem_label, columns=["sem_label"]))

In [ ]:
X = []

# Reading .bin files and adding to X_DF
for file in files_list:
    scan = np.fromfile(os.path.join(point_clouds_path, file), dtype=np.float32)
    scan = scan.reshape((-1, 4))

    # put in attribute
    points = scan[:, 0:3]    # get xyz
    remissions = scan[:, 3]  # get remission

    df_point_cloud = pd.DataFrame(points, columns=["x","y","z"])
    df_point_cloud["remissions"] = remissions
    X.append(df_point_cloud)

# print(X[0:2])
# print(list_labels[0:2])

del df_point_cloud # optimizar memoria

In [3]:
Y_concat = pd.concat(Y)
    
# df_list_labels


from matplotlib.ticker import FuncFormatter

# Crear el histograma
plt.figure(figsize=(16, 6))
#plt.style.use('ggplot')
plt.hist(Y_concat['sem_label'], 
         bins=np.arange(Y_concat['sem_label'].min(), Y_concat['sem_label'].max() + 2), 
         edgecolor='k', 
         alpha=1, 
         align='mid')

# Chatgpt code
# Configurar el formateador para el eje Y
#ax = plt.gca()
#formatter = FuncFormatter(lambda x, _: f"{int(x):,}")  # Formato con separadores de miles
#ax.yaxis.set_major_formatter(formatter)  # Aplicar el formateador

# Personalización del gráfico
plt.title("Histograma de Etiquetas Semánticas", pad=20)
plt.xlabel("Etiqueta Semántica", labelpad=10)
plt.ylabel("Frecuencia", labelpad=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Configurar las marcas del eje X en incrementos de 1
x_ticks = np.arange(Y_concat['sem_label'].min(), Y_concat['sem_label'].max() + 2, 1)
plt.xticks(x_ticks, fontsize=8)

# Ajustar padding de los x_ticks
plt.gca().tick_params(axis='x', pad=10)  # Aumenta el espacio entre los ticks y el eje

# Mostrar el gráfico
plt.show()

NameError: name 'Y' is not defined

In [ ]:
n_points_X = []

# Reading .bin files and adding to DF
for file in os.listdir(point_clouds_path):
    scan = np.fromfile(os.path.join(point_clouds_path, file), dtype=np.float32)
    scan = scan.reshape((-1, 4))

    # put in attribute
    points = scan[:, 0:3]    # get xyz
    remissions = scan[:, 3]  # get remissions

    n_points_X.append(len(points))

n_points_X = np.array(n_points_X)

MIN_POINTS_X = n_points_X[n_points_X.argmin()]
nube = n_points_X.argmin()

print(f"La nube con menos puntos tiene: {MIN_POINTS_X}, es la {nube}")

### Aplicar Permutación Aleatoria para establecer un _shape_ uniforme

In [ ]:
X_uniform = []
Y_uniform = []
    
for x, y in zip(X, Y):
    if len(x) >= MIN_POINTS_X:
        # Seleccionar 'MIN_POINTS_X' puntos aleatorios
        sampled_indices = np.random.choice(len(x), size=MIN_POINTS_X, replace=False)
        X_uniform.append(x.iloc[sampled_indices].reset_index(drop=True))
        Y_uniform.append(y.iloc[sampled_indices].reset_index(drop=True))
    else:
        raise ValueError(f"La nube tiene menos puntos ({len(x)}) que 'MIN_POINTS_X' ({MIN_POINTS_X}).")

In [ ]:
# Calcular la distancia máxima global
d_max = max(
    np.sqrt((df[['x', 'y', 'z']] ** 2).sum(axis=1)).max() for df in X_train
)

# Calcular la media y desviación estándar global de 'remissions'
all_remissions = pd.concat([df['remissions'] for df in X_train])
mean = all_remissions.mean()
std = all_remissions.std()

print(f"d_max: {d_max}, mean: {mean}, std: {std}")

In [ ]:
X_norm = []

for df in X:
    norm_df = df.copy()
    norm_df[['x','y','z']] = (df[['x','y','z']] / d_max)
    norm_df['remissions'] = (norm_df['remissions'] - mean) / std
    X_norm.append(norm_df)

In [ ]:
# Crear una ventana oculta de Tkinter
root = tk.Tk()
root.withdraw()  # Ocultar la ventana principal

# Ruta inicial
initial_path = Path("/Users/felixmaral/Desktop/TFG/datasets")

# Seleccionar la primera ruta
print("Selecciona la ruta para 'point_clouds_path'")
point_clouds_path_f = Path(filedialog.askdirectory(
    title="Selecciona la carpeta para 'point_clouds_path'",
    initialdir=initial_path
))

# Seleccionar la segunda ruta
print("Selecciona la ruta para 'labels_path'")
labels_path_f = Path(filedialog.askdirectory(
    title="Selecciona la carpeta para 'labels_path'",
    initialdir=initial_path
))

# Mostrar las rutas seleccionadas
print(f"Ruta seleccionada para point_clouds_path: {point_clouds_path}")
print(f"Ruta seleccionada para labels_path: {labels_path}")

In [ ]:
# Guardar cada DataFrame en un archivo
for i, df in enumerate(X_norm):
    # Define el nombre del archivo, por ejemplo: dataframe_0.csv
    file_name = f"dataframe_x_{count_x_train}.csv"  # Cambia a .parquet si prefieres parquet
    file_path = os.path.join(point_clouds_path_f, file_name)
    
    # Guardar el DataFrame como CSV
    df.to_csv(file_path, index=False)  # Usa index=False para omitir el índice
    print(f"Guardado: {file_path}")
    count_x_train += 1

for i, df in enumerate(Y):
    file_name = f"dataframe_y_{count_y_train}.csv"  # Cambia a .parquet si prefieres parquet
    file_path = os.path.join(labels_path_f, file_name)
    # Guardar el DataFrame como CSV
    df.to_csv(file_path, index=False)  # Usa index=False para omitir el índice
    print(f"Guardado: {file_path}")
    count_y_train += 1